In [43]:
import numpy as np

from keras.models import Sequential, load_model
from keras.layers.core import Dense, Activation, Dropout, TimeDistributedDense
from keras.layers.recurrent import LSTM

In [55]:
text = open('data/data_clean.txt', 'r').read().lower()
char_to_idx = {ch: i for (i, ch) in enumerate(sorted(list(set(text))))}
idx_to_char = {i: ch for (ch, i) in char_to_idx.items()}
vocab_size = 59
SEQ_LEN = 64
BATCH_SIZE = 16
BATCH_CHARS = len(text) // BATCH_SIZE
LSTM_SIZE = 256
LAYERS = 3

In [60]:
def read_batches(text):
    T = np.asarray([char_to_idx[c] for c in text], dtype=np.int32)
    X = np.zeros((BATCH_SIZE, SEQ_LEN, vocab_size))
    Y = np.zeros((BATCH_SIZE, SEQ_LEN, vocab_size))

    for i in range(0, BATCH_CHARS - SEQ_LEN - 1, SEQ_LEN):
        X[:] = 0
        Y[:] = 0
        for batch_idx in range(BATCH_SIZE):
            start = batch_idx * BATCH_CHARS + i
            for j in range(SEQ_LEN):
                X[batch_idx, j, T[start+j]] = 1
                Y[batch_idx, j, T[start+j+1]] = 1

        yield X, Y

def build_model(infer):
    if infer:
        batch_size = seq_len = 1
    else:
        batch_size = BATCH_SIZE
        seq_len = SEQ_LEN
    model = Sequential()
    model.add(LSTM(LSTM_SIZE,
                   return_sequences=True,
                   batch_input_shape=(batch_size, seq_len, vocab_size),
                   consume_less='cpu',
                   unroll=True,
                   stateful=True))

    #model.add(Dropout(0.2))
    for l in range(LAYERS - 1):
        model.add(LSTM(LSTM_SIZE, return_sequences=True, stateful=True))
        #model.add(Dropout(0.2))

    model.add(TimeDistributedDense(vocab_size))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='nadam')
    return model

def sample(sample_chars=256, primer_text='and the '):
    #model.reset_states()
    model.load_weights('karpathy_keras_char_rnn.h5')
    
    sampled = [char_to_idx[c] for c in primer_text]

    for c in primer_text:
        batch = np.zeros((1, 1, vocab_size))
        batch[0, 0, char_to_idx[c]] = 1
        model.predict_on_batch(batch)

    for i in range(sample_chars):
        batch = np.zeros((1, 1, vocab_size))
        batch[0, 0, sampled[-1]] = 1
        softmax = model.predict_on_batch(batch)[0].ravel()
        sample = np.random.choice(range(vocab_size), p=softmax)
        sampled.append(sample)

    print(''.join([idx_to_char[c] for c in sampled]))

In [66]:
model = build_model(infer=True)

sample(sample_chars=512, primer_text='bitch')

//anaconda/lib/python3.5/site-packages/keras/layers/core.py:1156: UserWarning: `TimeDistributedDense` is deprecated, And will be removed on May 1st, 2017. Please use a `Dense` layer instead.
  warnings.warn('`TimeDistributedDense` is deprecated, '


bitch on it for the felt pack in a night out of the amazing blender applying not end like a shot and a pencil on the towist is going to use a little bit of blending outsidiala and we're going to go for that blend out a little transition again to my entire harsh precise brush i love this brall but i am using i'm applying a light palette for my eyes already now i'm using the colour-pop lippie brown that is going to increase i'm actually gonna take the primer right now version to your eyebrows and the eyes for this
